Import Library

In [157]:
import pandas as pd
import torch
import accelerate
import chardet
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


In [158]:
with open('stat_problem.csv', 'rb') as file:
    result = chardet.detect(file.read())
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [159]:
data = pd.read_csv('stat_problem.csv', sep=',', encoding='cp1252')
#print(data.head()) Head hanya menampilkan 5 Index awal

In [160]:
data_clean = data.dropna(subset=['problem', 'answer'])

problem = data_clean['problem'].tolist()
res = data_clean['answer'].tolist()


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded = tokenizer(problem, padding=True, truncation=True, max_length=128, return_tensors='pt')

# tokenisasi Dataset
def tokenize_dataset(example) : 
    return tokenizer(
        example['problem'],
        truncation=True,
        padding=True,
    )


#split dataset menjadi data eval dan data train

train_text, val_text, train_label, val_label = train_test_split(
    problem,
    res,
    test_size=0.2 # 20% evaluasi
)


Tokenisasi DataSplit trainText dan evalText

In [161]:
train_encoding = tokenizer(train_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
eval_encoding = tokenizer(val_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

Buat dataset Torch Agar bisa di proses model BERT

In [162]:

class PeluangDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    

Buat train dan eval datset

In [163]:
train_dataset = PeluangDataset(encodings=encoded, labels=res)
eval_dataset = PeluangDataset(encodings=eval_encoding, labels=val_label)

In [164]:
# model pre Trained
model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [165]:

# set fine tuning models
training_args = TrainingArguments(
    output_dir='./output',
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=True
)

trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()

c:\Users\RPL SMKN 8 JEMBER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`